In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.set_printoptions(linewidth=1000, edgeitems=30)
pd.set_option('display.max_columns', 60)
pd.set_option('display.width', 2000)

In [ ]:
# Some StackOverflow magic to enable importing of local module
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from multiLevelCoSurrogates.local import base_dir

In [ ]:
records = ['_ei_records.csv', '_ucb_records.csv',]

print(os.listdir(base_dir))

df = pd.read_csv(base_dir+records[0], index_col='index')
df.head()

In [ ]:
grouped = df.groupby(by=['which_model', 'fidelity'])

n_cols = min(len(grouped), 3)
n_rows = int(np.ceil(len(grouped)/n_cols))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12,4*n_rows))

for ax, group in zip(axes.flatten(), grouped):
    name, sub_df = group
    sub_df = sub_df.groupby(by='iteration')[['mse_high', 'mse_low', 'mse_hier', 'mse_low_on_high']]
    means = sub_df.mean()
    stds = sub_df.std()
    means.plot(ax=ax)
    
    low_bounds, high_bounds = means.values.T - stds.values.T*1.96, means.values.T + stds.values.T*1.96
    for low, high, color in zip(low_bounds, high_bounds, ['C0', 'C1', 'C2', 'C3']):
        ax.fill_between(means.index, low, high, color=color, alpha=.2)

    ax.set_title('acq on {}, {} fidelity updated'.format(*name))

plt.tight_layout()
plt.show()

These plots show the raw MSE values (and 95% CI) for the three cases using the 'Expected Improvement' acquisition function. In the last plot, only the high fidelity model is updated, so the other values are constant.

In [ ]:
def plot_normalized(df, axes=None):

    grouped = df.groupby(by=['which_model', 'fidelity'])
    mse_names = ['mse_high', 'mse_low', 'mse_hier', 'mse_low_on_high']

    if axes is None:
        n_cols = min(len(grouped), 3)
        n_rows = int(np.ceil(len(grouped)/n_cols))
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(12,4*n_rows))
    
    for ax, group in zip(axes.flatten(), grouped):
        name, sub_df = group

        # Separate out the MSE values after initial training to use as reference
        num_iters = len(sub_df['iteration'].unique())
        first_rows = sub_df.loc[0::num_iters,mse_names].values

        # Get all values and reshape without copying s.t. the iterations are a separate dimension
        values = sub_df.loc[:,mse_names].values
        old_shape = values.shape
        new_shape = old_shape[0]//num_iters, num_iters, len(mse_names)
        values.shape = new_shape

        # Now we can use numpy's internal broadcasting for efficient division, and return the old shape afterwards
        new_values = values / first_rows[:,None,:]
        new_values.shape = old_shape

        # Finally the values are returned to the dataframe and plotted
        sub_df.loc[:,mse_names] = new_values
        sub_df = sub_df.groupby(by='iteration')[mse_names]
        means = sub_df.mean()
        stds = sub_df.std()
        
        means.plot(ax=ax)
        
        # low_bounds, high_bounds = means.values.T - stds.values.T*1.96, means.values.T + stds.values.T*1.96
        low_bounds, high_bounds = sub_df.quantile(.05).values.T, sub_df.quantile(.95).values.T
        
        for low, high, color in zip(low_bounds, high_bounds, ['C0', 'C1', 'C2', 'C3']):
            ax.fill_between(means.index, low, high, color=color, alpha=.2)
        
        ax.axhline(y=1.0, color='black', alpha=0.5)
        ax.set_title('acq on {}, {} fidelity updated'.format(*name))
#         ax.set_ylim([0,2])
        ax.set_yscale('log')

    plt.tight_layout()
    plt.show()

The following plots are all normalized to a starting MSE of '1.0', which is comparable as they are all initialized with just 3 high fidelity points and 5 low fidelity points.

In [ ]:
print(records[0])
plot_normalized(pd.read_csv(base_dir+records[0], index_col='index'))

In [ ]:
print(records[1])
plot_normalized(pd.read_csv(base_dir+records[1], index_col='index'))

In [ ]:
df = pd.read_csv(base_dir+records[0], index_col='index')
mse_names = ['mse_high', 'mse_low', 'mse_hier', 'mse_low_on_high']

# Separate out the MSE values after initial training to use as reference
num_iters = len(df['iteration'].unique())
first_rows = df.loc[0::num_iters,mse_names].values

# Get all values and reshape without copying s.t. the iterations are a separate dimension
values = df.loc[:,mse_names].values
old_shape = values.shape
new_shape = old_shape[0]//num_iters, num_iters, len(mse_names)
values.shape = new_shape

# Now we can use numpy's internal broadcasting for efficient division, and return the old shape afterwards
new_values = values / first_rows[:,None,:]
new_values.shape = old_shape

# Finally the values are returned to the dataframe and plotted
df.loc[:,mse_names] = new_values



b3hier = df[df['which_model'] == 'hierarchical']
b3hier = b3hier[b3hier['fidelity'] == 'both 3']
b3hier = b3hier.groupby(by='iteration')[['mse_high', 'mse_hier']]

highhigh = df[df['which_model'] == 'high']
highhigh = highhigh[highhigh['fidelity'] == 'high']
highhigh = highhigh.groupby(by='iteration')['mse_high']

fig = plt.figure(figsize=(12,9))
plt.plot(b3hier.mean()['mse_high'], label='both 3, hierarchical: mse_high')
plt.plot(b3hier.mean()['mse_hier'], label='both 3, hierarchical: mse_hier')
plt.plot(np.repeat(highhigh.mean().values, 3), label='high, high: mse_high')

low_bounds, high_bounds = b3hier.quantile(.05).values.T, b3hier.quantile(.95).values.T
plt.fill_between(b3hier.mean().index, low_bounds[0], high_bounds[0], color='C0', alpha=.2)
plt.fill_between(b3hier.mean().index, low_bounds[1], high_bounds[1], color='C1', alpha=.2)

low_bounds, high_bounds = highhigh.quantile(.05).values.T, highhigh.quantile(.95).values.T
plt.fill_between(np.arange(len(low_bounds))*3, low_bounds, high_bounds, color='C2', alpha=.2)

plt.axhline(y=1.0, alpha=0.33, color='black')
plt.legend(loc=0)
plt.yscale('log')
plt.show()



Here we focus on the best performing cases: acquisition function (expected improvement) on high fidelity only and hierarchical prediction.

In the above plot, the 'update high fidelity based on high fidelity acquisition function' plot has been stretched by a factor of 3 to match up with the number of high fidelity evaluations in the 'both every 3rd on hierarchical acquisition function' case.
It can be seen that, according to MSE, the hierarchical model (mse_hier) is actually better than the high fidelity model, with both of them ourperforming the single-fidelity (high, high) case.

The shaded areas show the 5th to 95th percentile of the MSE results. The spread is more varied for the bi-fidelity case at the start, but seems to become more stable towards the end of the optimization.